### Данные для ВКР

**Грузим данные**

no robots:

In [1]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/HuggingFaceH4/no_robots/" + splits["train"])

'HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/HuggingFaceH4/no_robots/resolve/main/data/train-00000-of-00001.parquet
Retrying in 1s [Retry 1/5].


beemo:

In [2]:
df2 = pd.read_parquet("hf://datasets/toloka/beemo/data/train-00000-of-00001.parquet")

'HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/toloka/beemo/resolve/main/data/train-00000-of-00001.parquet
Retrying in 1s [Retry 1/5].
'HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/toloka/beemo/resolve/main/data/train-00000-of-00001.parquet
Retrying in 2s [Retry 2/5].
'HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/toloka/beemo/resolve/main/data/train-00000-of-00001.parquet
Retrying in 1s [Retry 1/5].
'HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/toloka/beemo/resolve/main/data/train-00000-of-00001.parquet
Retrying in 1s [Retry 1/5].


**Проверяем `prompt id`**

check prompts example:

In [22]:
df.loc[df.prompt_id == df2.prompt_id[2], 'prompt']

7895    According to this statement from Chick-fil-A, ...
Name: prompt, dtype: object

In [23]:
df2.loc[df2.prompt_id == df2.prompt_id[2], 'prompt']

2    According to this statement from Chick-fil-A, ...
Name: prompt, dtype: object

**Как выглядит датасет**

In [26]:
df1 = df2.loc[:, ['prompt', 'model_output', 'human_output', 'human_edits']]

In [27]:
df1.head(3)

,prompt,model_output,human_output,human_edits
0,Who beat the Highland Society of Scotland in p...,Robert Armstrong beat the Highland Society of ...,Robert Armstrong released his dictionary in 18...,Robert Armstrong published a Scottish Gaelic d...
1,What family is the swordfish a part of?\nWhat ...,"Sure, I can answer your questions based on the...",Swordfish are in the family Xiphiidae.\nThe lo...,The swordfish is the only member of the family...
2,"According to this statement from Chick-fil-A, ...","According to the statement, Chick-fil-A's core...",Chick-fil-A says they do their best work throu...,"According to this statement, Chick-fil-A does ..."


**Предобработка**

Поменяем в формат `prompt | target | text`, колонка `variable` — таргет.

In [37]:
df3 = pd.melt(df1, id_vars=['prompt']).sort_values('prompt')

In [38]:
df3.head(10)

,prompt,variable,value
2294,- For what reasons does Google sometimes remov...,human_output,- Google sometimes removes content from its se...
4481,- For what reasons does Google sometimes remov...,human_edits,Google sometimes removes content from its sear...
107,- For what reasons does Google sometimes remov...,model_output,Google sometimes removes content from its sear...
166,- Who is currently serving as the UK Defence S...,model_output,The current Defence Secretary of the UK is Ben...
4540,- Who is currently serving as the UK Defence S...,human_edits,"The current UK Defence Secretary, a key figure..."
2353,- Who is currently serving as the UK Defence S...,human_output,- Ben Wallace is currently serving as the UK D...
2350,-Who now manages mascot Brutus Buckeye? \n-Whe...,human_output,-Ohio State’s cheerleading and spirit team man...
163,-Who now manages mascot Brutus Buckeye? \n-Whe...,model_output,Sure!\n\n**Who now manages mascot Brutus Bucke...
4537,-Who now manages mascot Brutus Buckeye? \n-Whe...,human_edits,The text does not specify who manages the masc...
87,1) Did Jerry Springer have a successful politi...,model_output,"Sure, here are the answers to your questions:\..."
